# <center> Marco-o1介绍&部署&评测

>主要内容图谱

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127182727360.png" style="zoom:60%;" />

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127135733192.png" style="zoom:60%;" />

## 为什么是o1

> 关于OpenAI-o1更多的技术报告：https://openai.com/index/learning-to-reason-with-llms/

Marco-o1，单看其模型名称不知大家是否发现一些端倪，`o1`指向的是现在最强的推理模型，也是OpenAI最新的产品--OpenAI-o1.

它具有以下特点：就像人类在回答困难问题之前会思考一段时间一样，o1 在尝试解决问题时使用思维链；它具有学会识别和纠正错误的能力；并将棘手的步骤分解为更简单的步骤；在当前方法不起作用时尝试不同的方法。

OpenAI-o1还有一个代号名字为“Strawberry”，其原因是它可以通过推理的方式解决经典大模型不会分词token识别的问题。

总之，OpenAI-o1作为大模型的新范式，具有极强的的推理能力和解决复杂问题的能力。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127114432362.png" width=60%></div>

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20241009223839616.png" alt="image-20241009223839616" style="zoom:33%;" />

尽管OpenAI-o1模型没有放出全部的训练信息，不过其透露出再训练阶段在推理阶段进行的投入是远超过其他所有大模型的这也是这个模型为什么称之为LRM--Large Reasoning Model的原因。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127115152007.png" width=80%></div>

正因如此，OpenAI-o1在进行回答推理的时候非常具有自己的特色，我们可以对比一下OpenAI-o1-mini和Open-4o的回答区别。

In [1]:
from openai import OpenAI
from IPython.display import display, Code, Markdown

In [4]:
api_key = 'your-openai-api-key'

In [3]:
# 实例化客户端
client = OpenAI(api_key=api_key, 
                base_url="https://ai.devtool.tech/proxy/v1")

In [4]:
# 调用 GPT-4o-mini 模型
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "你好，好久不见!"}
    ]
)

print(response.choices[0].message.content)

你好！很高兴见到你！有什么我可以帮助你的吗？


需要确认当前API-KEY权限是否可以调用o1模型：

In [5]:
models_list = client.models.list()

In [6]:
models_list.data

[Model(id='o1-preview', created=1725648897, object='model', owned_by='system'),
 Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'),
 Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'),
 Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='o1-preview-2024-09-12', created=1725648865, object='model', owned_by='system'),
 Model(id='gpt-4o-2024-08-06', created=1722814719, object='model', owned_by='system'),
 Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
 Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', 

In [13]:
response = client.chat.completions.create(
    model="o1-mini",
    messages=[
        {
            "role": "user", 
            "content": "写一个 Bash 脚本，将格式为 '[1,2],[3,4],[5,6]' 的字符串表示的矩阵转置，并以相同格式输出。"
        }
    ]
)

In [15]:
response

ChatCompletion(id='chatcmpl-AGSZ4hNjR4hQKZhJGwyVgsx1fI4pP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='当然，以下是一个完整的 Bash 脚本，它将按照您指定的格式将矩阵字符串进行转置。脚本包含详细的注释以帮助您理解每个步骤。\n\n```bash\n#!/bin/bash\n\n# 检查是否提供了输入字符串\nif [ $# -ne 1 ]; then\n    echo "用法: $0 \'[1,2],[3,4],[5,6]\'"\n    exit 1\nfi\n\ninput_str="$1"\n\n# 1. 替换 \'],[\' 为 \'|\', 并去除首尾的 \'[\' 和 \']\'\nprocessed_str=$(echo "$input_str" | sed \'s/\\]\\s*,\\s*\\[/|/g\' | sed \'s/\\[//g\' | sed \'s/\\]//g\')\n\n# 2. 使用 \'|\' 分割为行\nIFS=\'|\' read -r -a rows <<< "$processed_str"\n\n# 检查是否有至少一行\nif [ ${#rows[@]} -eq 0 ]; then\n    echo "输入格式错误。"\n    exit 1\nfi\n\n# 3. 将每行按 \',\' 分割为元素，并存储在二维数组中\ndeclare -a matrix\nnum_cols=0\nfor row in "${rows[@]}"; do\n    IFS=\',\' read -r -a elements <<< "$row"\n    matrix+=("${elements[@]}")\n    if [ ${#elements[@]} -gt $num_cols ]; then\n        num_cols=${#elements[@]}\n    fi\ndone\n\nnum_rows=${#rows[@]}\n\n# 4. 构建二维数组\ndeclare -A mat\n\nro

In [10]:
display(Markdown(response.choices[0].message.content))

以下是一个 Bash 脚本，它可以将格式为 `'[1,2],[3,4],[5,6]'` 的字符串表示的矩阵转置，并以相同格式输出。脚本使用 `sed` 和 `awk` 来处理字符串和执行矩阵转置。

```bash
#!/bin/bash

# 检查是否提供了输入参数
if [ $# -ne 1 ]; then
    echo "用法: $0 '[1,2],[3,4],[5,6]'"
    exit 1
fi

input="$1"

# 使用 sed 将矩阵行分割为多行，并移除方括号
# 然后使用 awk 进行转置
transposed=$(echo "$input" | \
    sed 's/\],[/\
/g' | sed 's/\[//g; s/\]//g' | \
    awk '
    {
        # 将每行按逗号分割成数组
        split($0, arr, ",")

        # 逐个存储到二维数组 a 中
        for (i=1; i<=length(arr); i++) {
            a[NR,i] = arr[i]
            if (i > max_cols) max_cols = i
        }
        if (NR > max_rows) max_rows = NR
    }
    END {
        # 遍历每一列，输出转置后的行
        for (i=1; i<=max_cols; i++) {
            line = ""
            for (j=1; j<=max_rows; j++) {
                line = line (j==1 ? "" : ",") a[j,i]
            }
            # 格式化输出，添加方括号和逗号分隔
            printf "%s%s", (i==1 ? "[" : "],[") , line
        }
        print "]"
    }')

# 输出转置后的矩阵
echo "$transposed"
```

### 使用方法

1. **保存脚本**

   将上述脚本内容保存为文件，例如 `transpose.sh`。

2. **赋予执行权限**

   ```bash
   chmod +x transpose.sh
   ```

3. **运行脚本**

   ```bash
   ./transpose.sh '[1,2],[3,4],[5,6]'
   ```

   **输出:**

   ```
   [1,3,5],[2,4,6]
   ```

### 脚本详解

1. **输入检查**

   ```bash
   if [ $# -ne 1 ]; then
       echo "用法: $0 '[1,2],[3,4],[5,6]'"
       exit 1
   fi

   input="$1"
   ```

   这部分检查是否提供了一个输入参数，如果没有，则提示用法并退出。

2. **预处理输入字符串**

   ```bash
   echo "$input" | \
       sed 's/\],[/\
   /g' | sed 's/\[//g; s/\]//g' | \
       awk '...'
   ```

   - `sed 's/\],[/\
   /g'`：将 `],[` 替换为换行符，实现将每个矩阵行分割到单独的一行。
   - `sed 's/\[//g; s/\]//g'`：移除所有的 `[` 和 `]` 符号，留下纯数字和逗号。

   例如，`'[1,2],[3,4],[5,6]'` 处理后变为：
   ```
   1,2
   3,4
   5,6
   ```

3. **使用 `awk` 进行转置**

   ```awk
   {
       split($0, arr, ",")

       for (i=1; i<=length(arr); i++) {
           a[NR,i] = arr[i]
           if (i > max_cols) max_cols = i
       }
       if (NR > max_rows) max_rows = NR
   }
   END {
       for (i=1; i<=max_cols; i++) {
           line = ""
           for (j=1; j<=max_rows; j++) {
               line = line (j==1 ? "" : ",") a[j,i]
           }
           printf "%s%s", (i==1 ? "[" : "],[") , line
       }
       print "]"
   }
   ```

   - **分割每行数据并存储到二维数组 `a` 中**
     - `split($0, arr, ",")`：将当前行按逗号分割成数组 `arr`。
     - `a[NR,i] = arr[i]`：将数据存储到二维数组 `a` 中，其中 `NR` 是行号，`i` 是列号。
     - `max_cols` 和 `max_rows` 用于记录矩阵的最大列数和行数。

   - **转置并格式化输出**
     - 外层循环遍历每一列 `i=1` 到 `max_cols`。
     - 内层循环遍历每一行 `j=1` 到 `max_rows`，构建转置后的每一行。
     - 使用 `printf` 和条件运算符添加必要的方括号和逗号分隔符。

4. **输出结果**

   将转置后的矩阵赋值给变量 `transposed`，然后使用 `echo` 输出。

### 示例

- **输入:**

  ```
  '[1,2],[3,4],[5,6]'
  ```

- **输出:**

  ```
  [1,3,5],[2,4,6]
  ```

- **输入:**

  ```
  '[1,2,3],[4,5,6]'
  ```

- **输出:**

  ```
  [1,4],[2,5],[3,6]
  ```

该脚本适用于任意大小的矩阵，只要输入格式正确，即每一行用方括号包裹，行与行之间用逗号分隔，元素之间用逗号分隔。

相比之下，GPT4o输出结果逻辑性和准确性就要弱很多：

In [11]:
# 调用 GPT-4o-mini 模型
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "写一个 Bash 脚本，将格式为 '[1,2],[3,4],[5,6]' 的字符串表示的矩阵转置，并以相同格式输出。"}
    ]
)

print(response.choices[0].message.content)

您可以使用以下 Bash 脚本来实现将格式为 '[1,2],[3,4],[5,6]' 的字符串表示的矩阵转置，并以相同格式输出:

```bash
#!/bin/bash

# 输入矩阵字符串
input_matrix='[1,2],[3,4],[5,6]'

# 去掉方括号并按逗号分割
matrix=$(echo $input_matrix | tr -d '[]' | tr ',' ' ')

# 将矩阵转换为数组
readarray -d ' ' -t rows <<< "$matrix"

# 确定行数和列数
num_rows=${#rows[@]}
num_cols=$(echo "${rows[0]}" | tr ' ' '\n' | wc -l)

# 创建转置后的矩阵
transposed=""

for ((i=0; i<num_cols; i++)); do
    row=""
    for ((j=0; j<num_rows; j++)); do
        value=$(echo ${rows[j]} | awk -v idx=$i '{print $((idx+1))}')
        row+="$value"
        if [[ $j -lt $((num_rows-1)) ]]; then
            row+=","
        fi
    done
    transposed+="[$row]"
    if [[ $i -lt $((num_cols-1)) ]]; then
        transposed+=","
    fi
done

# 输出转置后的矩阵
echo "$transposed"
```

### 使用说明：
1. 将上述脚本保存为 `transpose_matrix.sh`。
2. 在终端中为脚本添加可执行权限：
   ```bash
   chmod +x transpose_matrix.sh
   ```
3. 执行脚本：
   ```bash
   ./transpose_matrix.sh
   ```

该脚本将输入字符串 `input_matrix` 转置并输出。根据给定的示例，输出将是 `[[1,3,5],[2,4,6]]`。您可以根据需要修

In [12]:
display(Markdown(response.choices[0].message.content))

您可以使用以下 Bash 脚本来实现将格式为 '[1,2],[3,4],[5,6]' 的字符串表示的矩阵转置，并以相同格式输出:

```bash
#!/bin/bash

# 输入矩阵字符串
input_matrix='[1,2],[3,4],[5,6]'

# 去掉方括号并按逗号分割
matrix=$(echo $input_matrix | tr -d '[]' | tr ',' ' ')

# 将矩阵转换为数组
readarray -d ' ' -t rows <<< "$matrix"

# 确定行数和列数
num_rows=${#rows[@]}
num_cols=$(echo "${rows[0]}" | tr ' ' '\n' | wc -l)

# 创建转置后的矩阵
transposed=""

for ((i=0; i<num_cols; i++)); do
    row=""
    for ((j=0; j<num_rows; j++)); do
        value=$(echo ${rows[j]} | awk -v idx=$i '{print $((idx+1))}')
        row+="$value"
        if [[ $j -lt $((num_rows-1)) ]]; then
            row+=","
        fi
    done
    transposed+="[$row]"
    if [[ $i -lt $((num_cols-1)) ]]; then
        transposed+=","
    fi
done

# 输出转置后的矩阵
echo "$transposed"
```

### 使用说明：
1. 将上述脚本保存为 `transpose_matrix.sh`。
2. 在终端中为脚本添加可执行权限：
   ```bash
   chmod +x transpose_matrix.sh
   ```
3. 执行脚本：
   ```bash
   ./transpose_matrix.sh
   ```

该脚本将输入字符串 `input_matrix` 转置并输出。根据给定的示例，输出将是 `[[1,3,5],[2,4,6]]`。您可以根据需要修改 `input_matrix` 的值。

---

可以看到OpenAI-o1回复的信息更全面也更准确，对于用户输入的信息响应(意图识别)的也更好，这是其作为Large Reasoning Model推理类模型的优势，即专注于复杂的推理任务，通过强化学习技术提升其思维能力。与许多早期模型不同，o1 在回答之前会先“思考”，生成复杂的内部推理链。这种能力使 o1 非常适合处理需要深度逻辑处理的任务，例如编程竞赛、高级数学以及科学问题求解。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20241009224900159.png" alt="image-20241009224900159" style="zoom:33%;" />

不过OpenAI-o1性能如此强悍，是否有什么缺点呢？答案是有，它使用门槛很高且有限制，它限制了每周30次的问答使用上限，用户并不能很好的调用这个模型。但是关于这个模型的探索却不会停止，于是阿里的团队便研发出了同样类型的RLM —— Marco-o1.

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127135733192.png" alt="image-20241009224900159" style="zoom:33%;" />

## Marco-o1模型介绍

### 模型简介

Marco-o1是由阿里巴巴国际数字商务部MarcoPolo Team在hugging face平台发布全新开源大模型，它是由基础模型Qwen2：7B微调得来。模型性能主要是面向开放式解决方案的开放式推理模型。尤其是在**机器翻译任务**上表现优秀。

模型结构上：Marco-o1 大型语言模型 (LLM) 通**思维链 (CoT) 微调**、**蒙特卡洛树搜索 (MCTS)**、**反思机制**和**创新推理策略**进行优化——专为复杂的实际问题解决任务设计。

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127150811745.png" style="zoom:80%;" />

也就是说Marco-o1 是模仿OpenAI-o1的一次开源尝试，试图通过类似的手段去训练模型以期望达到优化的效果。

### 模型参数

Marco-o1是一个参数为7.62B ，精度BF16的轻量级模型。

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127153252658.png" alt="image-20241009224900159" style="zoom:80%;" />

> 该模型是基于Qwen2-7B微调训练得来，基于不同方式（CoT、MCTS的各种精度）其模型效果如上图所示。

Marco-o1模型在 AIME（美国数学奥林匹克竞赛） 和 CodeForces（编程与算法） 等测试中展现了强大的推理能力，Marco-o1 的目标是进一步突破结构化问题，追求更广泛的领域适应性，特别是在缺乏严格评估指标的场景下。通过结合思维链（CoT）微调、蒙特卡洛树搜索（MCTS）和推理动作策略等技术，Marco-o1 能够更加高效地应对复杂的问题解决任务。

- Marco-o1项目地址：https://github.com/AIDC-AI/Marco-o1
-  Marco-o1论文地址：https://arxiv.org/pdf/2411.14405

## Marco-o1模型部署&使用

> 本流程使用Ollama在Linux环境下通过命令行进行部署，如果对ollama安装和使用方式（如API调用、高级对话形式、模型管理等）有不熟悉的可以参考前面的视频内容进行学习。

### 本地部署硬件推荐

Linux系统：
1. 操作系统：推荐Ubuntu 22.04或更高版本，或其他支持的Linux发行版。
2. 显卡:推荐使用 NVIDIA GPU（如 T4、V100、A10 等），或 AMD GPU，在本流程中至少需要 6GB 的显存。
3. 内存：至少 16GB 的 RAM，在运行更大的模型的时候需要更多内存。（纯CPU推理情况）4. 硬盘空间：在每个模型的支持列表中会显示安装所需的空间大小，在本流程中最少5GB用于模型下载。

### 下载&更新Ollama

首先我们在命令行中输入以下信息实现进行Ollama的安装（如已经按照执行该命令会实现软件更新）。

下载Ollama的指令如下：

```
curl -fsSL https://ollama.com/install.sh | sh

```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241111104241685.png" width=80%></div>

下载完成后检测,如果返回版本号则说明成功下载：
```
ollama -v
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241120173953653.png" width=60%></div>

**下载模型**

在终端中执行命令 `ollama run marco-o1` ，即可下载该模型。模型下载完成后，会自动启动大模型，进入命令行交互模式，直接输入指令，就可以和模型进行对话了对应参数的模型的下载方式可以通过在Ollama官网查看到下载指令。

官网模型挂载地址: https://ollama.com/library/marco-o1

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125124339960.png" width=100%></div>

完成下载后会直接进入模型启动状态，如果退出或刷新界面，再次输入指令 `ollama run marco-o1` 即可启动对应模型。
使用Ollama进行推理marco-o1所需的计算资源仅需要不到6GB的显存。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125140900258.png" width=100%></div>

当命令行中返回>>>即可进行对话，当我进行简单测试的时候可以看到这个模型的回复风格与其他的确实大不相同。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125140915530.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125140940160.png" width=100%></div>

## 模型比较

下面我从几个维度进行marco-o1的性能测试：机器翻译、推理能力、数学能力、代码能力、开放性问题。

同时，我选择了Qwen2.5：7B作为对照组，理由是Qwen2.和Qwen2.5在模型结构上没有区别，唯独的变化是预训练时Qwen2.5的数据质量更高，进而在理论上Qwen2.5：7B的模型效果会比Qwen2：7B微调得来的marco-o1更好。（这里使用的都是instruct模型）

### 机器翻译

> 这个领域可以很好的落地于海外电商、字幕翻译、宣传等场景，翻译的最高境界是：信、达、雅。  即翻译要尽可能忠实于原文的内容和意义，同时内容要通顺易懂，语言要优美、得体，不仅要准确和流畅，还要符合目标语言的文化习惯和审美标准。

1. 这双鞋底部松软，踩起来有踩屎感，很舒服.

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125141920595.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126171941441.png" width=100%></div>

可以看到marco-o1在理解整句话之后进行翻译，做到了抓住句子核心思想。而Qwen2.5：7B只是讲句子进行了直译，所以`踩屎感`被理解成了不舒服的感觉，整个句子翻译下来矛盾。

---

2. 姐妹们，这个糯叽叽的真的很懂我们芋泥脑袋

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125143709542.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126172328648.png" width=100%></div>

marco-o1翻译的时候保留了原意（sisters、gooey thing、gets us），同时翻译效果比较符合英语语境比较直接的习惯，但是Qwen2.5：7B出现了错误翻译（lipstick），同时已经直接翻译现象严重（yummy heads）。

3. 识时务者为俊杰

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126173221393.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126173055837.png" width=80%></div>

marco-o1的翻译可以理解成：一个真正的领导者是能够理解时代的人，很好的把握住了`识时务`和`俊杰`的含义，Qwen2.5：7B在这里的的翻译也不差，将俊杰翻译成了有智慧的人。

4. 好风凭借力，送我上青云

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126173721192.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126173536729.png" width=100%></div>

在这句话中，两个模型都展现出了不错的翻译效果。

5. 这件衣服太炸了，我很喜欢它

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127104835210.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127104912736.png" width=100%></div>

在这句话中，两个模型都展现出了不错的翻译效果。

---

### 数字母

在大模型领域一直有这样一个难题，那就是很难数清一个单词里面对应的字母，即“strawberry”难题。这是由于大模型本身的机制导致的：LLM在识别字词的时候都是以token的形式进行理解的，一个token所涵盖的往往不是一整个单词（如下图所示），因此通常的大模型不能很好的理解单词的本质，进而无法正确数出“strawberry”中r的数量。而marco-o1在这个领域表现的就很好，这也是由于其推理能力的增强导致的。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127165745043.png" width=100%></div>

接下来进行一些简单的英语单词数字母测试，我们可以通过不同模型的分析过程看出它们在推理时的区别：

- How many Rs are in strawberry?

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125154905280.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126170852530.png" width=100%></div>

- How many 'e's are there in the word blueberry?

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125155120689.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126171011367.png" width=100%></div>

- How many a in the word of banana

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126171547520.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126171422328.png" width=100%></div>

可见在这个测试中marco-o1完胜Qwen2.5：7B。

### 推理能力

> 推理能力本身就是目前各大大模型研究团体正在突破的方向，具有更强推理能力的模型在面对复杂问题时才能展现出更好的适应能力。在想面对编程问题、数学问题的时候才会展现出更优的性能。

- 先进行一个小栗子测试，经典的9.11和9.8谁大：

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126174152927.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126174257620.png" width=100%></div>

看样子是描述的不够准确两个模型都理解成了比较日期大小，增加提示词，使其更准确的了解我们的问题，再进行推理回答。

- 进行数字比较，9.11和9.8哪个大

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126174235757.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126174342622.png" width=100%></div>

可以看到marco-o1很好的理解了这个问题的关键是在比较小数点后面的大小，进而在比较是按照百分数的形式进行推理，最后得出正确的答案。而Qwen2.5：7B直接给出答案，而后再进行分析，在分析推理的时候也是忽略了小数的作用直接比较了数字的大小（11-8）.

- 推理计算题

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175602462.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175602462.png" width=100%></div>

两个模型都通过正确的推理的到了正确的答案。

### 数学能力

- 计算未知数的值： 23x+87=4x*x

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125145516897.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125145813503.png" width=100%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175122462.png" width=100%></div>

对于这个中等难度的数学题，两个模型计算结果都正确.

- 证明极限：n趋于无穷的时候1/（n^2）为0

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125161952917.png" width=40%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125162006746.png" width=40%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125162637535.png" width=60%></div>

**Qwen2.5：7B**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175649646.png" width=60%></div>

对于这道高阶数学问题，两者的证明均合理且正确。marco-o1额外举了一个实例进行测试。

### 代码生成

- 为我用python写一段贪吃蛇代码，要求蛇是黄色的，食物是绿色的。

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125150027664.png" width=40%></div>

其生成的完整代码如下，我将其复制在编译器中进行测试，可以正常运行且按照我的要求将蛇设定为黄色，食物设定为绿色的。

```python
import pygame
import sys
from random import randrange

# Initialize Pygame
pygame.init()

# Define colors
yellow = (255, 255, 0)
green = (0, 255, 0)
black = (0, 0, 0)

# Set up the game window dimensions
width = 800
height = 600

# Create the display surface
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Snake Game")

# Clock for controlling the frame rate
clock = pygame.time.Clock()
fps = 15

# Snake settings
snake_size = 20
snake_speed = 20

# Food settings
food_size = 20

# Initialize snake position and direction
def init_snake():
    return [
        [100, 100],
        [80, 100],
        [60, 100]
    ], 'RIGHT'

snake, direction = init_snake()

# Function to generate food
def generate_food(snake):
    while True:
        x = randrange(0, width - food_size) // food_size * food_size
        y = randrange(0, height - food_size) // food_size * food_size
        if [x, y] not in snake:
            return [x, y]

food = generate_food(snake)

# Main game loop
while True:
    clock.tick(fps)
    
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and direction != 'DOWN':
                direction = 'UP'
            elif event.key == pygame.K_DOWN and direction != 'UP':
                direction = 'DOWN'
            elif event.key == pygame.K_LEFT and direction != 'RIGHT':
                direction = 'LEFT'
            elif event.key == pygame.K_RIGHT and direction != 'LEFT':
                direction = 'RIGHT'
    
    # Move the snake
    head = snake[0].copy()
    if direction == 'UP':
        head[1] -= snake_size
    elif direction == 'DOWN':
        head[1] += snake_size
    elif direction == 'LEFT':
        head[0] -= snake_size
    elif direction == 'RIGHT':
        head[0] += snake_size
    
    snake.insert(0, head)
    
    # Check for collision with food
    if snake[0] == food:
        food = generate_food(snake)
    else:
        snake.pop()
    
    # Check for collisions with walls or itself
    if (snake[0][0] < 0 or snake[0][0] >= width - snake_size or 
        snake[0][1] < 0 or snake[0][1] >= height - snake_size):
        pygame.quit()
        sys.exit()
    
    for segment in snake[1:]:
        if snake[0] == segment:
            pygame.quit()
            sys.exit()
    
    # Fill the background
    win.fill(black)
    
    # Draw food
    pygame.draw.rect(win, green, (food[0], food[1], food_size, food_size))
    
    # Draw snake
    for segment in snake:
        pygame.draw.rect(win, yellow, (segment[0], segment[1], snake_size, snake_size))
    
    pygame.display.flip()
```

实机效果演示：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241125150334483.png" width=60%></div>

**Qwen2.5：7B**

同样的问题交给Qwen2.5：7B，也会得到一段代码，但是其不能直接运行。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175513804.png" width=60%></div>

其生成的全部代码如下：

```python
import pygame
import random

# 初始化 Pygame
pygame.init()

# 设置窗口大小
width, height = 640, 480
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("黄色贪吃蛇游戏")

# 颜色定义
black = (0, 0, 0)
yellow = (255, 255, 0)
green = (0, 255, 0)

# 蛇和食物的初始位置
snake_pos = [[100, 50], [90, 50], [80, 50]]
food_pos = [random.randrange(1, (width//10)) * 10, random.randrange(1, (height//10)) * 
10]
food_spawn = True

# 方向
direction = 'RIGHT'
change_to = direction

clock = pygame.time.Clock()
snake_length = 3
snake_speed = 15

# 游戏循环
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                change_to = 'UP'
            if event.key == pygame.K_DOWN:
                change_to = 'DOWN'
            if event.key == pygame.K_LEFT:
                change_to = 'LEFT'
            if event.key == pygame.K_RIGHT:
                change_to = 'RIGHT'

    # 判断是否方向冲突
    if change_to == 'UP' and direction != 'DOWN':
        direction = 'UP'
    if change_to == 'DOWN' and direction != 'UP':
        direction = 'DOWN'
    if change_to == 'LEFT' and direction != 'RIGHT':
        direction = 'LEFT'
    if change_to == 'RIGHT' and direction != 'LEFT':
        direction = 'RIGHT'

    # 移动蛇
    if direction == 'UP':
        snake_pos[0][1] -= 10
    if direction == 'DOWN':
        snake_pos[0][1] += 10
    if direction == 'LEFT':
        snake_pos[0][0] -= 10
    if direction == 'RIGHT':
        snake_pos[0][0] += 10

    # 蛇身体增长机制
    snake_body = []
    for pos in snake_pos:
        snake_body.append(list(pos))
    if len(snake_body) > snake_length:
        del snake_body[0]

    # 检查是否吃到食物
    if snake_body[0] == food_pos:
        food_spawn = False
        snake_length += 1
    else:
        del snake_body[-1]

    # 重新生成食物
    if not food_spawn:
        food_pos = [random.randrange(1, (width//10)) * 10, random.randrange(1, 
(height//10)) * 10]
        food_spawn = True

    screen.fill(black)

    # 绘制蛇和食物
    for pos in snake_body:
        pygame.draw.rect(screen, yellow, pygame.Rect(pos[0], pos[1], 10, 10))
    
    pygame.draw.rect(screen, green, pygame.Rect(food_pos[0], food_pos[1], 10, 10))

    # 判断是否撞墙或自身
    if snake_body[0][0] < 0 or snake_body[0][0] >= width or snake_body[0][1] < 0 or 
snake_body[0][1] >= height:
        running = False

    for block in snake_body[:-1]:
        if snake_body[0] == block:
            running = False

    pygame.display.flip()
    clock.tick(snake_speed)

pygame.quit()

```

实际演示效果：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241126175436536.png" width=60%></div>

可见出现了一些语法错误。

### 开放性问题

- 巴黎的首都在哪，以及巴黎和柏林之间的距离。

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127111131904.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127175616466.png" width=60%></div>

对于这个问题，marco-o1回答的基本正确，不过更值得关注的是，对于一个足够开放的问题，marco-o1进行了思考并给出了足够多的解决办法：通过两个城市的经纬度来计算距离（由于没有合适的计算工具转而尝试其他方法），计算著名地标之间的距离，计算两个城市之间的列车或航班所需时间进行计算，最后根据自己的知识和能使用的工具进行汇总，最后给出答案。

- If I have 5 apples and give away 2, how many do I have left?

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127111306002.png" width=60%></div>

简单的推理问题，简单的推理解决出答案。

- 我们如何让世界更快乐

**marco-o1**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127104525253.png" width=60%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241127104515153.png" width=60%></div>

对于这更加开放式的问题，marco-o1确实进行了思考，并考虑到了三个元素'我们'、'世界'、'变快乐'，因此对于这个问题的答案不是照本宣科式的，而是建设性的提出了宏观上可以促进世界发展的方法。可见其在应对开放式问题时的优秀性能。

--- 

## 模型总评&展望


Marco-o1 模型展现出了卓越的推理能力，在翻译、推理和代码等领域均表现出色。这不仅体现在同参数规模（7B）的模型中，也真正展示了可直接应用的实际性能。在实际应用中，Marco-o1 可以作为中英翻译的有效工具，尤其擅长通过上下文理解词语之间的关系，从而实现“信达雅”的精准翻译。其优秀的推理能力使其特别适合应用于电商对外贸易、字幕翻译等场景。

此外Marco-o1作为一个轻量化的模型却展示出了更大参数模型才具备的性能，是与它的推理方式离不开的，但这也展示出了未来LLM发展的一个趋势，即LLMs组成Agents实现功能，在未来不需要一个巨大的模型来执行推理。许多参数专门用于记住事实，但这样的模型可以从知识中分离出推理，即一个小的 “推理核心” ，它知道如何调用浏览器和代码验证器等工具，而通过这些外挂工具以及它自身强大的推理能力便可以实现很多开放性的问题。


Marco-o1 实际上是对 OpenAI-o1 模型进行逆向工程的一个开源学习和模仿的成果，尽管如此，其效果仍然非常显著。这表明，未来会有更多类似的模型相继问世，这类模型不仅会在推理任务中表现出色，还将推动整个推理领域的技术进步。特别是作为一种新兴的 LRM（Large Reasoning Model） 类型，Marco-o1 展现了强大的跨领域推理能力，表明这类模型有着广泛的应用潜力。随着技术的不断发展，我们应持续关注这种推理模型的发展趋势，它们可能在自然语言处理、数据分析、自动推理等多个领域中发挥重要作用。因此，跟踪和研究 LRM 模型将对推动智能系统的进步与应用具有重要意义。